In [8]:
#参考https://github.com/NZbryan/NLP_bert
##去https://huggingface.co/bert-base-chinese 下载tf_model.h5到./pre_model/
import logging
logging.basicConfig(level=logging.ERROR)
from transformers import TFBertForSequenceClassification,BertTokenizer
import tensorflow as tf

import pandas as pd
from sklearn.model_selection import train_test_split

import sys 
sys.path.append("..") 
from utils import data_helper

import numpy as np

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
cn_class_dic={0: '家居', 1: '房产', 2: '教育', 3: '时尚', 4: '时政', 5: '科技', 6: '财经',7: '游戏',8: '娱乐',9: '体育'}
def getY(line):
    return cn_class_dic[line]

In [3]:
train_ds='../data/train/train_processed_v1.csv'
train_df=pd.read_csv(train_ds)[['cutted_content','label']]
train_df.columns=['content','label']

train_raw = train_df[['content','label']]

train_raw.columns=['text','y']
train_raw['label']=train_raw['y'].apply(getY)
train_raw.sample(10)

,text,y,label
1072,人民网 北京 9 月 8 日电 孙竞 9 月 7 日 北京市 52 万名 二至 四年级 学生...,2,教育
4379,1 基本面 2 月 马 棕 库存 下降 42MPOB 报告 利多 船调 机构 显示 本月 马...,6,财经
3761,9 月 11 日 广电总局 公示 8 月 全国 拍摄 制作 电视剧 备案 公示 显示 爱情 ...,8,娱乐
778,光明日报 记者 周仕兴 光明日报 通讯员 谢曼妮 一玉口 中国 一瓦顶 成家 都 说国 很大...,2,教育
3386,天涯 明月刀 手游 中 NPC 打好 关系 想要 NPC 打好 关系 肯定 送礼 一个 正确...,7,游戏
2540,一梦 江湖 一款 好玩 游戏 优秀 画风 优质 剧情 吸引 不少 玩家 很多 新手 玩家 不...,7,游戏
7212,做 期货 配资 心态 很 交易 习惯 带来 建议 期货市场 中 避开 陷阱 T0 交易 频繁...,6,财经
3593,2020 高校 招生 服务 光明 大 直播 光明日报 记者 苏雁 赵秋丽 冯帆 吴琳 刘勇 ...,2,教育
3234,期货 滚动 结算 滚动 结算 交易日 时间 内 结算 次数 不同于 商品 期货市场 每日 收...,6,财经
7535,8 月 6 日安兔 兔 官方 发布 2020 年 7 月份 价位 段 手机 性价比 排行榜 ...,5,科技


In [4]:
#建立模型
num_classes=10
learning_rate=2e-5

model = TFBertForSequenceClassification.from_pretrained('./pre_model', num_labels=num_classes)

# optimizer Adam recommended
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,epsilon=1e-08, clipnorm=1)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
print(model.summary())

Some layers from the model checkpoint at ./pre_model were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ./pre_model and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Model: "tf_bert_for_sequence_classification"
_____________________________________

In [3]:
def convert_example_to_feature(review):
  
  # combine step for tokenization, WordPiece vector mapping, adding special tokens as well as truncating reviews longer than the max length
	return tokenizer.encode_plus(review, 
	            add_special_tokens = True, # add [CLS], [SEP]
	            max_length = max_length, # max length of the text that can go to BERT
	            pad_to_max_length = True, # add [PAD] tokens
	            return_attention_mask = True, # add attention mask to not focus on pad tokens
		    truncation=True
	          )
# map to the expected input to TFBertForSequenceClassification, see here 
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(ds, limit=-1):
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit > 0):
        ds = ds.take(limit)
  
    for index, row in ds.iterrows():
        review = row["text"]
        label = row["y"]
        bert_input = convert_example_to_feature(review)
  
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)



def split_dataset(df):
    train_set, x = train_test_split(df, 
        stratify=df['label'],
        test_size=0.1, 
        random_state=42)
    val_set, test_set = train_test_split(x, 
        stratify=x['label'],
        test_size=0.5, 
        random_state=43)

    return train_set,val_set, test_set

In [6]:

#预处理数据

model_path = './pre_model/bert-base-chinese-vocab.txt' #模型路径，建议预先下载(https://huggingface.co/bert-base-chinese#)

max_length = 64
batch_size = 16
learning_rate = 2e-5
number_of_epochs = 1
num_classes = 10 # 类别数


# split data
train_data,val_data, test_data = split_dataset(train_raw)

In [7]:

# tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)
# train dataset
ds_train_encoded = encode_examples(train_data).shuffle(10000).batch(batch_size)
# val dataset
ds_val_encoded = encode_examples(val_data).batch(batch_size)
# test dataset
ds_test_encoded = encode_examples(test_data).batch(batch_size)


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [8]:
#训练模型
bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_val_encoded)
# evaluate test_set
print("# evaluate test_set:",model.evaluate(ds_test_encoded))

32/32 [==============================] - 47s 1s/step - loss: 0.3662 - accuracy: 0.9000
# evaluate test_set: [0.3661502003669739, 0.8999999761581421]


In [10]:
model.save_weights('bert_10')

In [3]:
#建立模型
num_classes=10
learning_rate=2e-5

new_model = TFBertForSequenceClassification.from_pretrained('./pre_model', num_labels=num_classes)

# optimizer Adam recommended
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,epsilon=1e-08, clipnorm=1)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
new_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
print(new_model.summary())

Some layers from the model checkpoint at ./pre_model were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ./pre_model and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Model: "tf_bert_for_sequence_classification"
_____________________________________

In [4]:
new_model.load_weights('bert_10')

In [5]:
#在测试集上输出结果
test_ds='../data/test_data.csv'
test_df=pd.read_csv(test_ds)[['content']]
## 大于4k 长度的比例
#print(len(test_df[test_df['content'].str.len()>4000]['content'])/len(test_df))
## 截断到 4k
idxs=test_df.loc[test_df['content'].str.len()>4000].index
#print(test_df.loc[test_df['content'].str.len()>4000].index)
test_df.loc[test_df['content'].str.len()>4000,'content']=test_df.loc[test_df['content'].str.len()>4000,'content'].apply(lambda x:x[:4000].rsplit('。',1)[0])

#删除除字母,数字，汉字以外的所有符号删除除字母,数字，汉字以外的所有符号
test_df['cleaned_content']=test_df['content'].apply(data_helper.remove_punctuation)

In [6]:
def encode_test(line):
    return np.array(tokenizer.encode(line,max_length = max_length))

In [9]:
model_path = './pre_model/bert-base-chinese-vocab.txt'
tokenizer = BertTokenizer.from_pretrained(model_path)
test_encode=test_df['cleaned_content'].apply(encode_test)


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [10]:
test_encode.size

20000

In [50]:
predict_y=[]
for i in range(test_encode.size):
    #tmp=np.array(new_model.predict([test_encode[i]])[0][0])
    predict_y.append(new_model.predict([test_encode[i]])[0][0])

In [51]:
predict_y=np.array(predict_y)
y_pred1 = predict_y.argmax(axis = 1)

array([7, 9, 0, 0, 7, 5, 9, 3, 4, 8])

In [ ]:
#{0: '家居', 1: '房产', 2: '教育', 3: '时尚', 4: '时政', 5: '科技', 6: '财经',7: '游戏',8: '娱乐',9: '体育'}
#7类，分别为：财经、房产、家居、教育、科技、时尚、时政
#10个类别:财经、房产、家居、教育、科技、时尚、时政、游戏、娱乐、体育。
#{0: '可公开', 1: '低风险', 2: '中风险', 3: '高风险'}
#rank_label对应关系:0{0,8,9} 1{2,3,7} 2{1,5} 3{4,6}
def reflect_rank(line):
    res=-1
    if(line==0 or line==8 or line==9):
        res=0
    elif(line==2 or line==3 or line==7):
        res=1
    elif(line==1 or line==5):
        res=2
    elif(line==4 or line==6):
        res=3
    return res

cn_class_dic={0: '家居', 1: '房产', 2: '教育', 3: '时尚', 4: '时政', 5: '科技', 6: '财经',7: '游戏',8: '娱乐',9: '体育'}
cn_rank_dic={0: '可公开', 1: '低风险', 2: '中风险', 3: '高风险'}

def cn_class_label(line):
    return cn_class_dic[line]

def cn_rank_label(line):
    return cn_rank_dic[line]

In [ ]:
#id class_label rank_label

test_pr=pd.DataFrame(y_pred1)
test_pr.columns = ['class_label']
test_pr['rank_label']=test_pr['class_label'].apply(reflect_rank)
test_pr['class_label']=test_pr['class_label'].apply(cn_class_label)
test_pr['rank_label']=test_pr['rank_label'].apply(cn_rank_label)#id class_label rank_label

test_pr=pd.DataFrame(y_pred1)
test_pr.columns = ['class_label']
test_pr['rank_label']=test_pr['class_label'].apply(reflect_rank)
test_pr['class_label']=test_pr['class_label'].apply(cn_class_label)
test_pr['rank_label']=test_pr['rank_label'].apply(cn_rank_label)
test_pr.to_csv('submission_bert_10.csv',index_label='id')